<a href="https://colab.research.google.com/github/yuren32/-/blob/master/03_%E6%95%99%E5%B8%AB%E3%81%82%E3%82%8A%E5%AD%A6%E7%BF%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#教師あり学習（Supervised Learning）Part 1



### 　教師あり学習とは

教師あり学習＝説明変数（入力）と目的変数（出力）を対応付ける関数を作ること．

【教師あり学習の手順】

1.  データ（説明変数と目的変数を含む学習データ）を用意する
2.  データの整理・前処理，特徴抽出をする
3.  データを訓練（training）用と検証（test）用に分割し，訓練用データを使って学習モデルのパラメータをチューニングする（＝学習，モデル構築）
4.  検証用データを使って， 構築したモデル（学習後のモデル）の性能を評価する

###使用するライブラリの読み込み

In [ ]:
# データ加工・処理・分析ライブラリ
import numpy as np
import numpy.random as random
from pandas import Series, DataFrame
import pandas as pd
import scipy as sp

# 機械学習ライブラリ
import sklearn

# 可視化ライブラリ
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

# 小数第3位まで表示
%precision 3

##重回帰

【テストデータセット】：https://archive.ics.uci.edu/ml/datasets/automobile

【対象問題】自動車の属性（大きさや種類，エンジンなど）からその自動車の価格を予測する

####■データの読み込み

In [ ]:

import requests, zipfile
import io

# 自動車価格データを取得
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data'
res = requests.get(url).content

# 取得したデータをDataFrameオブジェクトとして読み込み
auto = pd.read_csv(io.StringIO(res.decode('utf-8')), header=None)

# データの列にラベルを設定
auto.columns =['symboling','normalized_losses','make','fuel_type' ,'aspiration','num_of_doors',
                            'body_style','drive_wheels','engine_location','wheel_base','length','width','height',
                            'curb_weight','engine_type','num_of_cylinders','engine_size','fuel_system','bore',
                            'stroke','compression_ratio','horsepower','peak_rpm','city_mpg','highway_mpg','price']

In [ ]:
#全部表示設定
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

display(auto)
print(auto.shape)
display(auto.info())



####■データの整理，前処理
*  ここでは簡単に，peak_rpm，width，heightという３つの説明変数から，priceという目的変数を予測するというモデルを作成する
*  実際は，priceを予測するのにどのような説明変数を使えばよいかを考えることが重要となる（特徴量選択）
*  特徴量選択の方法には，「変数増加法（前進的選択法）」「変数減少法（後退的選択法）」などがある．また，「AIC（赤池情報量基準）」「BIC（ベイズ情報量基準）」「MDL」や「汎化誤差」などの評価基準を用いた方法や，組み合わせ最適化問題を解くアルゴリズム（遺伝的アルゴリズムなど）を利用する方法などがある．しかし，いずれの方法も絶対的に有効とは言えないため，様々な観点から検討する必要がある．
*  また，データセットの値をそのまま使うのではなく，何かしらの数学的理論や手法に基づいて，新たな数値を算出して用いる場合もある

In [ ]:
#peak-rpm，width，height, priceだけにする
auto2 = auto[['peak_rpm','width','height','price']]
display(auto2)

In [ ]:
#各カラムに「?」が何個あるかカウント
auto2.isin(['?']).sum()

In [ ]:
# '?'をNaNに置換し，NaNがある行を削除
auto2 = auto2.replace('?', np.nan).dropna()
print(auto2.shape)

In [ ]:
#型をチェック
print(auto2.dtypes)


In [ ]:
#priceがobjectなので，数値型に変換
auto2 = auto2.assign(peak_rpm=pd.to_numeric(auto2.peak_rpm))
auto2 = auto2.assign(price=pd.to_numeric(auto2.price))

#型を再チェック
print(auto2.dtypes)

In [ ]:
#相関を見てみる
auto2.corr()

*  重回帰分析においては，相関の高い変数を使用すると多重共線性（multi-collonearity）が生じる
*  多重共線性＝相関の高い変数の回帰係数の分散が大きくなり，係数の優位性が失われてしまう現象
*  一般的に，重回帰に限らず，相関の強い変数（特徴量）は，どれか1つあればで良い

In [ ]:
#データを　説明変数X　と　目的変数y　に分ける
X = auto2.drop('price', axis=1)
y = auto2['price']

print(X)
print(y)

####■学習（モデル構築）

*   重回帰においては，説明変数を入力すると，目的変数を正しく出力できるような演算となるように，モデル係数，残差を推定すること









In [ ]:
#訓練データ，テストデータ分割
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from sklearn.linear_model import LinearRegression

#モデルの初期化と学習
mlr_model = LinearRegression()
mlr_model.fit(X_train, y_train)

#モデルの概要
display(mlr_model)
print('---------------')

#構築したモデルのパラメータ（回帰係数と残差）の値
print(pd.Series(mlr_model.coef_, index=X.columns))
print('---------------')
print(mlr_model.intercept_)

####■評価（モデル検証）

In [ ]:
# 構築したモデルで予測
y_pred = mlr_model.predict(X_test)

#実際の値と予測値の確認
print(y_test)
print(y_pred)

In [ ]:
# 訓練データに対するスコア（決定係数）の算出
result = mlr_model.score(X_train, y_train)
print(result)

In [ ]:
# テストデータに対するスコア（決定係数）の算出
result = mlr_model.score(X_test, y_test)
print(result)

###【演習課題】他の特徴量を選択して，回帰（予測）の精度を上げてみよう．

ただし，ここでは問題を簡単にするために，値が数値の特徴量（UCI機械学習レポジトリーで「Type」の項目が「Integer」か「Continuous」のもののうち，値が数値のもの）の中から選択すること．

In [ ]:
#自動車データをmyautoにコピー
myauto = auto.copy()

In [ ]:
#コピーしたmyautoを使って，以下に処理を書いてみよう



##ロジスティック回帰

■重回帰を参考に，以下の問題を自分で解いてみよう
* 【テストデータセット】重回帰と同じ自動車データセット
* 【問題設定】任意の説明変数を6つ選び，makeを目的変数として識別する．ただし，makeは，日本メーカーと海外メーカーの2値に変換して，識別する．

*  日本メーカー =  honda, isuzu, mazda, mitsubishi, nissan, subaru, toyota
*  海外メーカー = alfa-romero, audi, bmw, chevrolet, dodge, jaguar, mercedes-benz, mercury,  peugot, plymouth, porsche,
renault, saab, volkswagen, volvo


■ヒント
*  ロジスティック回帰は，「from sklearn.linear_model import LogisticRegression」を用いる
* make変数をどのように二値化するかがポイント
1.   PandasのDataFrameの「replace」を使う方法
2.   PandasのDataFrameの「loc」や「at」を使う方法
3.   lambda式を使う方法


*  各説明変数は，それぞれの単位や大きさが異なっているため，モデルの学習が値の大きな変数に引っ張られ値の小さな変数の影響が小さくなる懸念がある．それを避けるには，前処理として説明変数を標準化する．標準化することで識別精度が向上する可能性がある．

In [ ]:
#二値化のヒント

df = pd.DataFrame([["Ichiro", 33, 175.3, 61.1],
                   ["Jiro", 25, 185.3, 93.1],
                   ["Saburo", 22, 170.3, 57.9],
                   ["Shiro", 17, 165.1, 54.5],
                   ["Goro", 15, 155.3, 43.2],
                   ["Rokuro", 15, 155.3, 43.2]
                   ])
df.columns = ['Name', 'Age', 'Tall', 'Weight']
display(df)


In [ ]:
#replaceの使用例
df['Name'].replace('Ichiro', '一郎', inplace=True)

display(df)

In [ ]:

#ワンホット表現という{0,1}にする
pd.get_dummies()

#名義尺度（勝手な数字に割り当て）
pd.factorize()


In [ ]:

url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data'
res = requests.get(url).content

# 取得したデータをDataFrameオブジェクトとして読み込み
auto = pd.read_csv(io.StringIO(res.decode('utf-8')), header=None)

# データの列にラベルを設定
auto.columns =['symboling','normalized_losses','make','fuel_type' ,'aspiration','num_of_doors',
                            'body_style','drive_wheels','engine_location','wheel_base','length','width','height',
                            'curb_weight','engine_type','num_of_cylinders','engine_size','fuel_system','bore',
                            'stroke','compression_ratio','horsepower','peak_rpm','city_mpg','highway_mpg','price']


In [ ]:
#factorizeの方法
auto2 = auto.copy()
auto2 = auto2.replace('?', np.nan).dropna()

col_name = ['fuel_type', 'aspiration', 'num_of_doors', 'body_style', 'drive_wheels', 'engine_location', 'engine_type', 'num_of_cylinders', 'fuel_system']
for c in col_name:
    auto2[c] = auto2[c].factorize()[0]

display(auto2)


In [ ]:
#get_dummiesの方法
auto2 = auto.copy()
auto2 = auto2.replace('?', np.nan).dropna()


col_name = ['fuel_type', 'aspiration', 'num_of_doors', 'body_style', 'drive_wheels', 'engine_location', 'engine_type', 'num_of_cylinders', 'fuel_system']


auto3 = pd.get_dummies(auto2, columns=col_name)
display(auto3)
